In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from keras.optimizers import SGD
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
%matplotlib inline
from keras.wrappers.scikit_learn import KerasClassifier
import sklearn
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score
from scipy import interp
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import cross_val_score

In [4]:
#loading data
dataset = np.load('data.row/ASD2.npz') #Dataset ready in numpy array (removing background, resizing, and transforming into grayscale)
X = dataset['X']
y = dataset['y']

print(X.shape)
print(y.shape)

(3103, 224, 224, 3)
(3103, 1)


In [6]:
def GetModel():
    resnet50 = tf.keras.applications.ResNet50(weights='imagenet', include_top=True , input_tensor=None, input_shape=None, pooling=None)

    x = resnet50.layers[-2].output
    output = Dense(units=1, activation='sigmoid')(x)
    model = Model(inputs=resnet50.input, outputs=output)
    for layer in model.layers[175:]:
      layer.trainable = True
    for layer in model.layers[:175]:
      layer.trainable = False


    opt = SGD(lr=0.001,momentum=0.99,decay=0.01,nesterov=False)
    #opt = Adam(learning_rate=0.0001,beta_1=0.99,beta_2=0.99,epsilon=0.1,amsgrad=False,name="Adam",)

    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [7]:
#Hyperparameters
nfolds = 3
nEpochs = 30
nBatch= 16
#inputDim = X.shape[1] # Count of features

In [6]:
#kfold = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=1).split(X, y) #Cross validation
#for train, test in kfold:
 # print(X[train].shape, X[test].shape ,y[train].shape, y[test].shape )

(2068, 224, 224, 3) (1035, 224, 224, 3) (2068, 1) (1035, 1)
(2069, 224, 224, 3) (1034, 224, 224, 3) (2069, 1) (1034, 1)
(2069, 224, 224, 3) (1034, 224, 224, 3) (2069, 1) (1034, 1)


In [13]:
def train_cross_validate(folds = 4):
    histories = []
    models = []
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 4)
    kfold = StratifiedKFold(n_splits=nfolds, shuffle=True, random_state=1).split(X, y) #Cross validation
    for train, test in kfold:
        
        model = GetModel()()
        history = model.fit(X[train], y[train], validation_split= 0.2, epochs=nEpochs, batch_size=nBatch, verbose=2,callbacks = [ early_stopping])
       
        models.append(model)
        histories.append(history)
        
    return histories, models

In [15]:
histories, models = train_cross_validate()#تا اینجا به شکل تابع بودن رو تست کردم که نتونستم ارور رو رفع کنم

ValueError: ignored

In [7]:
model = GetModel()#از اینجا به بعد فقط از تابعی که برای رزنت تعریف کردم استفاده کردم و خروجی میده اما نه چیزی که من می خوام 

In [8]:
hist =model.fit(X[train], y[train], validation_split=0.2, epochs=nEpochs, batch_size=nBatch, verbose=2)

Epoch 1/30
104/104 - 7s - loss: 0.8384 - accuracy: 0.6356 - val_loss: 4.3076 - val_accuracy: 0.0000e+00
Epoch 2/30
104/104 - 6s - loss: 0.7731 - accuracy: 0.6242 - val_loss: 1.3345 - val_accuracy: 0.0000e+00
Epoch 3/30
104/104 - 6s - loss: 0.6947 - accuracy: 0.6731 - val_loss: 2.8887 - val_accuracy: 0.0000e+00
Epoch 4/30
104/104 - 6s - loss: 0.7327 - accuracy: 0.6332 - val_loss: 1.8756 - val_accuracy: 0.0000e+00
Epoch 5/30
104/104 - 6s - loss: 0.6071 - accuracy: 0.7432 - val_loss: 0.6987 - val_accuracy: 0.4227
Epoch 6/30
104/104 - 6s - loss: 0.6009 - accuracy: 0.7372 - val_loss: 2.6824 - val_accuracy: 0.0000e+00
Epoch 7/30
104/104 - 6s - loss: 0.6040 - accuracy: 0.7456 - val_loss: 1.7501 - val_accuracy: 0.0000e+00
Epoch 8/30
104/104 - 6s - loss: 0.5529 - accuracy: 0.7444 - val_loss: 1.4977 - val_accuracy: 0.0000e+00
Epoch 9/30
104/104 - 6s - loss: 0.5716 - accuracy: 0.7444 - val_loss: 2.1483 - val_accuracy: 0.0000e+00
Epoch 10/30
104/104 - 6s - loss: 0.6088 - accuracy: 0.7082 - val_los

In [10]:
keras_clf = KerasClassifier(GetModel)#این قسمت رو دوباره برای کافولد استفاده کردم ولی نتیجه من نسبت به کدی که دیدم عجیب بود.،باید 3 تا عدد می داد ولی فقط یک عدد چاپ کرده و بقیه صفر هستند ،چرا؟
accuracies = cross_val_score(estimator=keras_clf, X=X[test], y=y[test], cv=3)

print(accuracies)

11/11 [==============================] - 1s 110ms/step - loss: 14.0395 - accuracy: 0.0000e+00
[0.         0.78550726 0.        ]


In [9]:
#for i, layer in enumerate(model.layers):
    #print(i, layer.name, layer.trainable)

In [11]:
for layer in model.layers[165:]:
    layer.trainable = True
for layer in model.layers[:165]:
    layer.trainable = False

In [12]:
opt = SGD(lr=0.001,momentum=0.95,decay=0.01,nesterov=False)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
hist =model.fit(X[train], y[train], validation_split=0.2, epochs=nEpochs, batch_size=nBatch, verbose=2)

Epoch 1/30
104/104 - 8s - loss: 0.5389 - accuracy: 0.7492 - val_loss: 1.4069 - val_accuracy: 0.0097
Epoch 2/30
104/104 - 7s - loss: 0.5116 - accuracy: 0.7523 - val_loss: 0.7546 - val_accuracy: 0.4082
Epoch 3/30
104/104 - 7s - loss: 0.4900 - accuracy: 0.7644 - val_loss: 0.1004 - val_accuracy: 0.9976
Epoch 4/30
104/104 - 7s - loss: 0.4630 - accuracy: 0.7843 - val_loss: 0.3938 - val_accuracy: 0.8792
Epoch 5/30
104/104 - 7s - loss: 0.4480 - accuracy: 0.7994 - val_loss: 0.8908 - val_accuracy: 0.3575
Epoch 6/30
104/104 - 7s - loss: 0.4550 - accuracy: 0.7849 - val_loss: 0.4340 - val_accuracy: 0.8309
Epoch 7/30
104/104 - 7s - loss: 0.4419 - accuracy: 0.7970 - val_loss: 0.3238 - val_accuracy: 0.9106
Epoch 8/30
104/104 - 7s - loss: 0.4404 - accuracy: 0.8018 - val_loss: 0.0598 - val_accuracy: 0.9952
Epoch 9/30
104/104 - 7s - loss: 0.4231 - accuracy: 0.8079 - val_loss: 1.6280 - val_accuracy: 0.1812
Epoch 10/30
104/104 - 7s - loss: 0.4221 - accuracy: 0.8169 - val_loss: 0.0681 - val_accuracy: 0.9928

In [14]:
score = model.evaluate(X[test], y[test], verbose=0 )
print(score)

[1.6486421823501587, 0.6324951648712158]


In [15]:
y_pred = model.predict(X[test])

AUC = roc_auc_score(y[test], y_pred.round())
print(AUC)

0.5454545454545454
